# Recommedation model

In [1]:
# Load libraries
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 
from scipy.stats import mannwhitneyu
from sklearn.preprocessing import StandardScaler
import os
import time

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# utils import
from fuzzywuzzy import fuzz

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id="627a2a038c2c4bfda963e47db3b99a1a", client_secret="c2c46473dadc486da9670277e1179bf0")

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

sp.trace=False

In [3]:
# for recommendation system we will consider only liked songs
liked_songs = pd.read_csv("liked_songs.csv")
liked_songs_1 = pd.read_csv("C:/Users/nehas/OneDrive/Desktop/liked_songs.csv")
liked_songs_2 = pd.read_csv("C:/Users/nehas/OneDrive/Desktop/liked.csv")

In [4]:
df = pd.concat([liked_songs,liked_songs_1,liked_songs_2], axis = 0).reset_index(drop=True)
df.shape

(6738, 18)

Again we are going to drop unwanted columns and duplicate values

In [5]:
ids = df[['id']]

In [6]:
df.drop(columns=['uri', 'track_href', 'analysis_url','type','id'],inplace=True)

In [7]:
# KNN with euclidean distance


# Features have different ranges; re-scale them first
# Initializing scaler
scaler = StandardScaler()
# Separating features from target ('like')
# Standardizing features
all_songs_df_std = pd.DataFrame(StandardScaler().fit_transform(df))
all_songs_df_std.columns = df.columns
print(all_songs_df_std.shape)
all_songs_df_std.head()
all_songs_df_std.index = ids['id']

(6738, 13)


In [8]:
# Get artist and song name from song_id
def get_playlist_songs(df):
    idx_list = []
    for song_id in df.index:
        artist_name = [feat['name'] for feat in sp.track(song_id)['album']['artists']]
        song_name = sp.track(song_id)['name']
        name = ' '.join(map(str, artist_name))  + ':' + song_name       
        idx_list.append(name)
    return idx_list

In [37]:
"""artist_songs_list = get_playlist_songs(all_songs_df_std)
"""

'artist_songs_list = get_playlist_songs(all_songs_df_std)\n'

In [ ]:
"""# Add to df index
all_songs_df_std.index = artist_songs_list
all_songs_df_std.head()"""

In [ ]:
"""all_songs_df_std.to_csv("songs_artist.csv")"""

In [38]:
all_songs_df_std= pd.read_csv("songs_artist.csv")

In [19]:
all_songs_df_std=all_songs_df_std.set_index('Unnamed: 0')

In [26]:
song_to_idx = {song: i for i, song in enumerate(list(all_songs_df_std.index))}

In [27]:
# transform matrix to scipy sparse matrix
song_user_mat_sparse = csr_matrix(all_songs_df_std.values)

print(f"Sparse matrix:{song_user_mat_sparse}")

Sparse matrix:  (0, 0)	-1.3476373798981045
  (0, 1)	1.4706690391302326
  (0, 2)	0.4768150007404325
  (0, 3)	1.048195543773315
  (0, 4)	-1.2376020878366538
  (0, 5)	0.3277632930161038
  (0, 6)	-1.0439411917488877
  (0, 7)	-0.1771843602592614
  (0, 8)	-0.3795536211640843
  (0, 9)	-1.0583626936661097
  (0, 10)	0.3067533997725489
  (0, 11)	-0.2216953802726657
  (0, 12)	0.1674318468086723
  (0, 13)	-1.3962564367553263
  (0, 14)	1.00242060368338
  (0, 15)	-1.0440647660576834
  (1, 0)	-0.55353531135696
  (1, 1)	1.707915541276215
  (1, 2)	-1.2080858488343171
  (1, 3)	1.4433700778954603
  (1, 4)	-1.2376020878366538
  (1, 5)	0.5611931835398571
  (1, 6)	-1.0222472295617524
  (1, 7)	-0.1925339165221156
  (1, 8)	0.1626370764932776
  :	:
  (11993, 7)	-0.1925339165221156
  (11993, 8)	-0.7503622017458436
  (11993, 9)	-0.5998689915400746
  (11993, 10)	1.0704701084088746
  (11993, 11)	-0.2108692831974027
  (11993, 12)	0.1674318468086723
  (11993, 13)	1.1999450760148087
  (11993, 14)	-0.997585241489964
 

In [40]:
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
# fit
model_knn.fit(song_user_mat_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [41]:
def fuzzy_matching(mapper, fav_song, verbose=True):
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map song title name to index of the song in data

    fav_song: str, name of user input song
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_song.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]



def make_recommendation(model_knn, data, mapper, fav_song, n_recommendations):
    """
    return top n similar song recommendations based on user's input song


    Parameters
    ----------
    model_knn: sklearn model, knn model

    data: song-user matrix

    mapper: dict, map song title name to index of the song in data

    fav_song: str, name of user input song

    n_recommendations: int, top n recommendations

    Return
    ------
    list of top n similar song recommendations
    """
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input song:', fav_song)
    idx = fuzzy_matching(mapper, fav_song, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_song))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[idx], dist))

In [46]:
my_favorite = 'Imagine Dragons'

make_recommendation(
    model_knn=model_knn,
    data=song_user_mat_sparse,
    fav_song=my_favorite,
    mapper=song_to_idx,
    n_recommendations=5)

You have input song: Imagine Dragons
Found possible matches in our database: ['Imagine Dragons:Demons', 'Imagine Dragons:Natural', 'Imagine Dragons:Wrecked', 'Imagine Dragons:Thunder', 'Imagine Dragons:Bad Liar', 'Imagine Dragons:Believer', "Imagine Dragons:It's Time", 'Imagine Dragons:Follow You', 'Imagine Dragons:Radioactive', 'Imagine Dragons:Whatever It Takes']

Recommendation system start to make inference
......

Recommendations for Imagine Dragons:
1: The Beatles:The Night Before - Remastered, with distance of 0.1383057958348184
2: Hanif Shaikh Vijay Lama:Bekarar, with distance of 0.13539280948938714
3: AURORA:The River, with distance of 0.13088975713061168
4: Ellie Goulding:Burn, with distance of 0.1280178036917885
5: Various Artists:Meri Maa, with distance of 0.12607322857661585
